# NOMAD API Authentication

This notebook demonstrates how to authenticate with the NOMAD API using the utilities from the `nomad_api` package.

In [1]:
# Import the authentication module from the nomad_api package
import os
import sys
from pathlib import Path

# Add parent directory to path if needed
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))

from nomad_api.auth import authenticate, OASIS_OPTIONS, get_token, verify_token

## Available NOMAD Instances

The authentication module defines several NOMAD instances you can connect to:

In [2]:
# Display available NOMAD instances
print("Available NOMAD instances:")
for name, url in OASIS_OPTIONS.items():
    print(f"- {name}: {url}")

# Set the instance you want to use
selected_instance = "SE Oasis"  # Change this to your desired instance
base_url = OASIS_OPTIONS[selected_instance]
print(f"\nUsing instance: {selected_instance} ({base_url})")

Available NOMAD instances:
- SE Oasis: https://nomad-hzb-se.de/nomad-oasis/api/v1
- CE Oasis: https://nomad-hzb-ce.de/nomad-oasis/api/v1
- Sol-AI Oasis: https://nomad-sol-ai.de/nomad-oasis/api/v1

Using instance: SE Oasis (https://nomad-hzb-se.de/nomad-oasis/api/v1)


## Authentication Methods

There are several ways to authenticate with the NOMAD API:

1. **Auto method**: Tries to use token from environment, then username/password from environment
2. **Token method**: Uses the NOMAD_CLIENT_ACCESS_TOKEN environment variable
3. **Password method**: Uses username and password provided directly

Below are examples of each method:

### Method 1: Automatic Authentication (Recommended)

This method tries various authentication sources in sequence.

In [3]:
try:
    token, user_info = authenticate(base_url, method="auto")
    print(f"Successfully authenticated as: {user_info.get('name', user_info.get('username', 'Unknown user'))}")
    print(f"Your user ID: {user_info.get('user_id', 'Unknown')}")
    print(f"\nToken (first 20 chars): {token[:20]}...")
except ValueError as e:
    print(f"Authentication failed: {e}")

Successfully authenticated as: Paolo Graniero
Your user ID: 36705df2-2d25-4421-a247-2995ef5cdcc0

Token (first 20 chars): eyJhbGciOiJSUzI1NiIs...


In [4]:
from nomad_api.client import NomadClient
from nomad_api.auth import authenticate, OASIS_OPTIONS

# Choose a NOMAD instance
selected_instance = "SE Oasis"  # Change this to your desired instance
base_url = OASIS_OPTIONS[selected_instance]

# Get authentication token
token, user_info = authenticate(base_url, method="auto")

In [5]:
client = NomadClient(base_url, token)

In [8]:
# General structure:
# client.make_request(method, endpoint, params=None, json_data=None, timeout=10)

headers = {"Authorization": f"Bearer {token}"}
response = client.make_request("get", "uploads", params=headers)

In [10]:
response["data"]

[{'process_running': False,
  'current_process': 'process_upload',
  'process_status': 'SUCCESS',
  'last_status_message': 'Process process_upload completed successfully',
  'errors': [],
  'warnings': [],
  'complete_time': '2024-10-08T19:53:21.743000',
  'upload_id': '8DtT7SHGQ_S_B11jz5sAWw',
  'upload_name': 'Python Scripts',
  'upload_create_time': '2023-09-06T15:08:01.971000',
  'main_author': 'e2796594-6a85-4a0b-a7ae-5862505c24d7',
  'coauthors': ['33aef548-d8e1-4c91-acc6-8618576580f7',
   'c8c3cb2e-8b51-41f9-bdc8-469762de3f5e',
   '67803968-dc2d-4e6b-96dc-cbbe617eb375',
   'b45bc41a-3676-4b54-909e-0ec091279be0',
   'e9e2df2f-5332-417e-9ae2-c5f1fc253678',
   '672a0e0f-671d-4745-9032-b4d4f2e75927',
   '36705df2-2d25-4421-a247-2995ef5cdcc0',
   'b35a6768-df0b-4183-822b-6b490c4bbacb',
   'aeaab123-3634-4f0d-acac-03c4acc4e393'],
  'coauthor_groups': [],
  'reviewers': [],
  'reviewer_groups': [],
  'writers': ['e2796594-6a85-4a0b-a7ae-5862505c24d7',
   'c8c3cb2e-8b51-41f9-bdc8-469762